In [2]:
import os
import numpy as np
import pandas as pd
import multiprocessing
import sys
import json
import tqdm
os.chdir("/home/ec2-user/SageMaker/foodi-ml/notebooks/")

In [2]:
from importlib import reload

In [3]:
import utils_aws

In [19]:
utils_aws

<module 'utils_aws' from '/home/ec2-user/SageMaker/foodi-ml/notebooks/utils_aws.py'>

In [4]:
utils_aws = reload(utils_aws)

# Configurations

In [5]:
conf = {
    "S3_BUCKET": 'test-bucket-glovocds',
    "S3K_imgs": 'artifacts/002/',
    "S3_file_samples": 'training_data.csv',
    "pth_dwn_samples": '/home/ec2-user/SageMaker/data/samples',
    "pth_dwn_images": '/home/ec2-user/SageMaker/data/images',
    "pth_vocab": '/home/ec2-user/SageMaker/foodi-ml/.vocab_cache/foodiml_vocab.json',
    "pth_dataset_json": '/home/ec2-user/SageMaker/data/foodiml/dataset_foodiml.json',
}

# AWS

In [6]:
# AWS classes
aws_con = utils_aws.AWSConnector(conf["S3_BUCKET"])
awstools = utils_aws.AWSTools(aws_con)
aws_basics = utils_aws.AWSBasics(conf["S3_BUCKET"])

# List cities

In [7]:
l_cities = awstools.create_list_cities(conf['S3K_imgs'])
print(l_cities)

['BCN', 'CUG']


# Download samples CSV

In [17]:
l_cities = ['CUG']

In [18]:
l_cities = awstools.downloading_city_csv(
    l_cities=l_cities,
    s3_key_prefix=conf['S3K_imgs'],
    csv_name=conf['S3_file_samples'],
    local_folder=conf['pth_dwn_samples'],
    verbose=True
)

Key artifacts/002/CUG/training_data.csv not found in S3
Removing from l_cities city CUG


In [21]:
conf['S3_file_samples']

'training_data.csv'

In [16]:
conf['pth_dwn_samples']

'/home/ec2-user/SageMaker/data/samples'

In [20]:
city = "CUG"

In [22]:
# Get the prefix of the S3 csv name
local_file_name = f"{city}.csv"

# Get full local path
local_file_path = os.path.join(conf['pth_dwn_samples'], local_file_name)

# S3 key of the .csv
s3k_file_path = os.path.join(conf['S3K_imgs'], city, conf['S3_file_samples'])

# Download csv
success = aws_basics.download_obj(
    s3_key=s3k_file_path,
    destination=local_file_path
)

Key artifacts/002/CUG/training_data.csv not found in S3


In [23]:
success

False

# Read and concatenate CSV

In [9]:
# Get all csv we downloaded
l_csv = os.listdir(conf['pth_dwn_samples'])

# Impose only .csv
l_csv = [file_name for file_name in l_csv if file_name.endswith(".csv")]

In [14]:
conf['pth_dwn_samples']

'/home/ec2-user/SageMaker/data/samples'

In [10]:
# Concatenation to a single dataframe
samples = pd.DataFrame()

In [11]:
# Read and concatenate each city dataframe to samples
for city_csv_file in tqdm.tqdm(l_csv):
    path_csv = os.path.join(conf['pth_dwn_samples'], city_csv_file)
    df_city_csv = pd.read_csv(path_csv)
    df_city_csv.insert(loc=0,
                       column="city", 
                       value = city_csv_file.split(".csv")[0])
    samples = pd.concat([samples, df_city_csv])

0it [00:00, ?it/s]


In [4]:
## NEW UPDATES PONÇ FINAL REVIEWS
# Now we have the entire dataframe, we read it and dump it into a parquet in the same way david was doing
import pandas as pd
import os
conf = {
    "S3_BUCKET": 'test-bucket-glovocds',
    "S3K_imgs": 'artifacts/002/',
    "S3_file_samples": 'glovo-foodi-ml-dataset.csv',
    "pth_dwn_samples": '/home/ec2-user/SageMaker/dataset/',
    "pth_dwn_images": '/home/ec2-user/SageMaker/dataset/dataset/',
    "pth_vocab": '/home/ec2-user/SageMaker/foodi-ml/.vocab_cache/foodiml_vocab.json',
    "pth_dataset_json": '/home/ec2-user/SageMaker/data/foodiml/dataset_foodiml.json',
}

In [ ]:
samples = pd.read_csv(os.path.join(conf['pth_dwn_samples'], conf["S3_file_samples"]))

In [5]:
samples.to_parquet(os.path.join(conf['pth_dwn_samples'], "samples.parquet"), engine = "pyarrow")

# Download images specified in samples CSV

In [5]:
samples = pd.read_parquet(os.path.join(conf['pth_dwn_samples'], "samples.parquet"), engine = "pyarrow")

# Create vocabulary

## 1) Create sentences

In [6]:
samples = pd.read_parquet(os.path.join(conf['pth_dwn_samples'], "samples.parquet"), engine = "pyarrow")

In [10]:
samples["sentence"] = \
    np.where(samples["product_name"], samples["product_name"], "") + " " + \
    np.where(samples["collection_section"], samples["collection_section"], "") + " " + \
    np.where(samples["product_description"], samples["product_description"], "")

samples["sentence"] = samples["sentence"].str.lower()

In [12]:
sentences = samples["sentence"].values

### 1.1) Fit tokenizer

In [13]:
os.chdir("/home/ec2-user/SageMaker/foodi-ml/")

In [15]:
from retrieval.data.tokenizer import Tokenizer

ModuleNotFoundError: No module named 'torch'

In [72]:
tokenizer = Tokenizer(vocab_path=None, download_tokenizer=True)

[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [73]:
tokenizer.vocab.word2idx

{'<pad>': 0, '<unk>': 1, '<start>': 2, '<end>': 3}

In [74]:
vocab = tokenizer.fit(sentences)

100%|██████████| 8011/8011 [00:01<00:00, 7386.66it/s]


In [75]:
# Saving vocabulary
tokenizer.save(conf['pth_vocab'])

### 1.2) Creating dataset json

In [76]:
def clean_special_characters(sentence):
    # Clean isolated characters
    remove_chars = ['.', '-', ')', '(','[', ']','{','}','?','!','€','$','#','@','*', '/']
    for char in remove_chars:
        if char in sentence:
            sentence.remove(char)
    return sentence

In [78]:
samples_dataset = {
    "images": [],
    "dataset": "foodiml"
}

In [79]:
sample_size = samples.shape[0]
samples_train = int(sample_size * 0.7)

In [80]:
print("Samples train: ", samples_train)
print("Samples valid: ", sample_size - samples_train)

Samples train:  5607
Samples valid:  2404


In [11]:
samples.columns

Index(['Unnamed: 0', 'Unnamed: 0.1', 'country_code', 'city_code', 'store_name',
       'product_name', 'collection_section', 'product_description', 'subset',
       'hash', 'aux_store', 'HIER', 's3_path', 'sentence'],
      dtype='object')

In [8]:
samples.subset.value_counts()

train    2021210
test      433117
val       433117
Name: subset, dtype: int64

In [81]:
for i, row in tdm(samples.iterrows()):
    raw_sentence = row["sentence"]
    filename = row["s3_path"].split("/")[-1]
    sentence_tokens = tokenizer.split_sentence(raw_sentence)
    sentence_json = {}
    sentence_json["imgid"] = i
    sentence_json["sentences"] = [
        {
            "tokens": sentence_tokens,
            "raw": raw_sentence,
            "imgid": i
        }
    ]
    sentence_json["split"] = row["subset"]
    sentence_json["filename"] = filename
    samples_dataset["images"].append(sentence_json)

8011it [00:02, 3946.59it/s]


In [82]:
# Save dataset_foodiml.json 
with open(conf['pth_dataset_json'], "w") as f:
    json.dump(samples_dataset, f)